In [1]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.8 MB/s eta 0:00:00


In [2]:
import tiktoken

In [3]:
enc = tiktoken.get_encoding('cl100k_base')
assert enc.decode(enc.encode('hello there')) == 'hello there'

enc = tiktoken.encoding_for_model('gpt-4')

In [4]:
assert enc.decode(enc.encode('hello there')) == 'hello there'

In [5]:
enc.encode('hello there')

[15339, 1070]

In [6]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-09 13:26:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2023-10-09 13:26:04 (103 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [7]:
with open('input.txt', 'r') as f:
    text = f.read()

In [8]:
text_tokens = enc.encode(text)

In [9]:
len(text_tokens)

301829

In [10]:
enc.n_vocab

100277

In [11]:
enc.decode([text_tokens[0]])

'First'

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
data = torch.tensor(text_tokens, dtype=torch.long)

In [14]:
data.shape

torch.Size([301829])

In [15]:
lr = 3e-4

In [16]:
len(text.split(' ')), len(text)

(169893, 1115394)

In [17]:
chars = sorted(list(set(text.split(' '))))
vocab_size = len(chars)

In [18]:
chars[2]

'\nWas'

In [19]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
def encode(s): return [stoi[c] for c in s]
def decode(l): return ' '.join([itos[x] for x in l])

In [20]:
data = torch.tensor(encode(text.split(' ')), dtype=torch.long)

In [21]:
data[:10]

tensor([ 1455,   957, 39874, 29614,  5949, 16628, 18572, 24432, 34050, 34057])

In [22]:
decode(encode(text.split(' ')[:10]))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst'

In [23]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [25]:
def get_decay_matrix(dim, gamma):
    d = torch.ones(dim)
    d = torch.tril(d) # lower triangular matrix

    for index, head in enumerate(d):
        g = gamma[index]
        for idx, x in enumerate(torch.tril(head)):
            for idy, y in enumerate(x):
                if idx >= idy:
                    head[idx][idy] = g ** (idx-idy)
    return d

In [26]:
!pip install -q einops
import einops
from einops import rearrange, reduce, repeat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00


In [27]:
class ChunkwiseRetention(nn.Module):
    def __init__(self, chunk_size, num_head, block_size):
        super().__init__()
        self.key = nn.Linear(n_embed, chunk_size*num_head, bias=False)
        self.query = nn.Linear(n_embed, chunk_size*num_head, bias=False)
        self.value = nn.Linear(n_embed, chunk_size*num_head, bias=False)
        self.gamma = 1.0-2.0**(-5-torch.arange(0, num_head))
        self.decay_mask = get_decay_matrix((num_head, block_size, block_size), self.gamma)
        self.chunk_decay = self.gamma
        self.gn = nn.GroupNorm(1, num_head)
        self.num_head = num_head
        self.chunk_size = chunk_size

    def forward(self, x, past_kv):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        k = rearrange(k, ('b t (h c) -> b h t c'), t=T, h=self.num_head, c=self.chunk_size)
        q = rearrange(q, ('b t (h c) -> b h t c'), t=T, h=self.num_head, c=self.chunk_size)
        v = rearrange(v, ('b t (h c) -> b h t c'), t=T, h=self.num_head, c=self.chunk_size)

        retention = q @ k.transpose(-1, -2)
        retention = retention * self.decay_mask
        inner_retention = retention @ v

        past_kv = repeat(past_kv, 'n q v -> B n q v', B=B)
        pb, pn, pq, pv = past_kv.shape

        padding = torch.zeros(pb, pn, pq, self.chunk_size)
        past_kv = past_kv + padding

        dm = repeat(self.decay_mask, 'h c d -> B h c d', B=B)
        pp = q @ past_kv
        cross_retention = pp.transpose(-1, -2) @ dm
        cross_retention = cross_retention.transpose(-1, -2)

        retention = inner_retention + cross_retention

        current_kv = self.gamma.view(self.num_head, 1, 1) * past_kv + (k.transpose(-1, -2) @ v)
        output = self.gn(retention.transpose(-1, -2))
        output = rearrange(output, 'b c h t -> b t (c h)')
        return output, current_kv.mean(dim=0)

In [28]:
class GatedMultiScaleRetention(nn.Module):
    def __init__(self, chunk_size, num_head, block_size):
        super().__init__()
        self.wg = nn.Linear(n_embed, n_embed, bias=False)
        self.act = nn.SiLU()
        self.y = ChunkwiseRetention(num_head=n_head, chunk_size=n_embed//n_head, block_size=block_size)
        self.wo = nn.Linear(n_embed, n_embed, bias=False)
        self.past = torch.zeros(num_head, chunk_size, chunk_size)

    def forward(self, x):
        wgx = self.wg(x)
        wgx = self.act(wgx)
        y, past = self.y(wgx, self.past)
        y = wgx * y
        return self.wo(y)

In [29]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4*n_embed),
            nn.GELU(),
            nn.Linear(4*n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [30]:
class Block(nn.Module):
    def __init__(self, n_embed, n_head, block_size):
        super().__init__()
        self.sa_head = GatedMultiScaleRetention(num_head=n_head, chunk_size=n_embed//n_head, block_size=block_size)
        self.ffw = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_head(self.ln1(x))
        x = x + self.ffw(self.ln2(x))
        return x

In [31]:
class RetNet(nn.Module):
    def __init__(self, block_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head, block_size=block_size) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            b, s = idx.shape
            bk = min(s, block_size)
            idx_cond = torch.cat((torch.zeros(b, block_size-bk, dtype=int), idx), dim=1)[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [32]:
def get_batch(split, batch_size):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [35]:
batch_size = 16
block_size = 32
max_iters = 5000
eval_iterval = 100
lr = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.0

In [36]:
model = RetNet(block_size=block_size)
xb, yb = get_batch('train', batch_size=batch_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
logits, loss = model(xb, yb)

In [37]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size=batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [38]:
for iter in range(max_iters):
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: Train Loss {losses['train']:.4f}, Val Loss {losses['val']:.4f}")

    xb, yb = get_batch('train', batch_size=batch_size)

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

Step 0: Train Loss 11.0299, Val Loss 11.0290
Step 100: Train Loss 9.5518, Val Loss 9.6659
Step 200: Train Loss 8.3693, Val Loss 8.6311
Step 300: Train Loss 8.1147, Val Loss 8.4785
Step 400: Train Loss 8.0443, Val Loss 8.4210
Step 500: Train Loss 8.0098, Val Loss 8.4514
Step 600: Train Loss 7.9541, Val Loss 8.4678
Step 700: Train Loss 7.9416, Val Loss 8.4939
Step 800: Train Loss 7.9074, Val Loss 8.5544
Step 900: Train Loss 7.8954, Val Loss 8.5651
Step 1000: Train Loss 7.8639, Val Loss 8.5639
Step 1100: Train Loss 7.8472, Val Loss 8.5571
Step 1200: Train Loss 7.8231, Val Loss 8.6143
Step 1300: Train Loss 7.8032, Val Loss 8.5881
Step 1400: Train Loss 7.7807, Val Loss 8.6421
Step 1500: Train Loss 7.7787, Val Loss 8.6343
Step 1600: Train Loss 7.7459, Val Loss 8.6390
Step 1700: Train Loss 7.7198, Val Loss 8.6767
Step 1800: Train Loss 7.7155, Val Loss 8.6750
Step 1900: Train Loss 7.6969, Val Loss 8.7068
Step 2000: Train Loss 7.6798, Val Loss 8.6861
Step 2100: Train Loss 7.6476, Val Loss 8.736

In [39]:
context = torch.tensor([encode('thou art kneel before king'.split(' '))], dtype=torch.long)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))

thou art kneel before king with my dear thou dost be dragon: XI:
And prosecute
'Gainst buzz if silent: there joy at foretold in consul.

MENENIUS:
Fie, thought love the pent-up up hateful consort'st O, we Master death,
Their out what doth you am for't:
All with him to her Was you have been led will hinder ear thou, your own.
For of commanded able LEWIS Bolingbroke?
Gardener, one bloody for daughter:
The hearten 's you to save my part, sweat the rest,
Yet ancient boast
As the tub.

LUCIO:
Why, of wretched for thy part, put ELIZABETH:
Ah, and there threats!

YORK:
Will thou hast with thine with he,
'That his thing seek You'll him hurt in thy mind; oath?
As makes the greater issue
Of are than news?

Second thou hair
Shall, Pray straight;
No pardons,
That OF AUMERLE:
Where grumbling will steal and friend
And confess,
Great thought this part you take remember done?

SICINIUS:
The these good made
Good master more for water-drops!
Good that ever Bolingbroke.

SIR good signior, on this taverns